In [1]:
# Restarta python caso necessário.
# from databricks.sdk.runtime import *
# dbutils.library.restartPython()

# Import de Funções e Libs

As bibliotecas e funções utilizadas ao longo do código devem ser adicionadas aqui

In [2]:
import numpy as np
import pandas as pd
import requests
import json
import sherlock
import regras_politica as rp


pd.set_option("display.max_columns", None)

### Testes Modulares - Referente ao tópico 3 - Desenho de Execução do Processo (Modelo de Priorização de Score e GH)

### Geração de Massa

In [3]:
#Gerando a massa modelo passando o caminho do JSON como nome da variável
base_entrada = pd.DataFrame({
                             'solicitante.numeroDocumento':['123456789'],
                             'solicitante.dataAdmissaoFopag':["2002-11-05T17:18:43.888Z"],
                             'solicitante.dataRegistroPrimeiroContrato':["2002-11-05T17:18:43.888Z"],
                             'solicitante.dataRegistro':["2002-11-05T17:18:43.888Z"],
                             'solicitante.flagMAT30d':[0],
                             'solicitante.flagMAT35d':[0],
                             'solicitante.flagMAT60d':[0],
                             'solicitante.flagMAT90d':[0],
                             'solicitante.fonteRenda':['XYZ'],
                             'solicitante.flagPortabilidade':[0],
                             'solicitante.rendaLiquidaPicpay':[10000],
                             'solicitante.consumerID':[12345678987654321],
                             'solicitante.scoreInternoLongo':[999],
                             'solicitante.scoreInterno':[999],
                             'solicitante.investimentoTotal':[0],
                             'solicitante.flagInvestimentoPrincipalidade':[0],
                             'solicitante.flagPrincipalidadeAll':[0],
                             'solicitante.flagPrincipalidade':[0],
                             'solicitante.mesFlagPrincipalidade':[0],
                             'solicitante.flagAllowlist':["-99999"],
                             'solicitante.flagBlocklist':["-99999"],
                             'solicitante.scoreInternoCurto':[959],
                             'solicitante.scoreExterno':[959],
                             'solicitante.flagCartaoCreditoContratado':[0],
                             'solicitante.flagLimiteGarantidoContratado':[0],
                             
            
                             'solicitante.ghAppcardsBlendInterno':["B"],
                             'solicitante.scoreAppcardsBlendInternoOnline':[650],
                             'solicitante.ghAppcardsBlendExterno':["C"],
                             'solicitante.scoreAppcardsBlendExternoOnline':[650],
                             'solicitante.ghBlendIntExt':["E"],
                             'solicitante.scoreBlendIntExtOnline':[500],
                             'solicitante.ghSerasa':["F"],
                             'solicitante.scoreSerasaAlOnline':[450]
    
                            })

In [4]:
#Criação das massas com casos forçados


# Criacao Modelos de Priorização - ADICIONADO NOVA VERSAO

massa_modelosPriorizacao = sherlock.Gera_Massa(base_entrada)
massa_modelosPriorizacao.gera_cenario('solicitante.ghAppcardsBlendInterno',['B','missing','C','0'])
massa_modelosPriorizacao.gera_cenario('solicitante.scoreAppcardsBlendInternoOnline',['650',-1,0,-10])
massa_modelosPriorizacao.gera_cenario('solicitante.ghAppcardsBlendExterno',['D','missing','C','0'])
massa_modelosPriorizacao.gera_cenario('solicitante.scoreAppcardsBlendExternoOnline',[500,-1,0,-10])
massa_modelosPriorizacao.gera_cenario('solicitante.ghBlendIntExt',['E','missing','C','0'])
massa_modelosPriorizacao.gera_cenario('solicitante.scoreBlendIntExtOnline',[100,0,-1])
massa_modelosPriorizacao.gera_cenario('solicitante.ghSerasa',['F','missing','C','0'])
massa_modelosPriorizacao.gera_cenario('solicitante.scoreSerasaAlOnline',[1000,0,-1,-99999])


massa_modelosPriorizacao.set_bom_arqv()
massa_modelosPriorizacao.converte_massa()

--------------------------------------
Payload setado com sucesso
--------------------------------------
Conversao concluida com sucesso


### Execução Modular e Geração de CSV

In [5]:
listaResultado = rp.execucaoModular(massa_modelosPriorizacao.get_massa_json(),rp.CriacaoModelosPriorizacao)

In [6]:
# Renomeação das colunas para o cabeçalho do RMA
massa_modelosPriorizacao.rename_cabecalho()

In [7]:
#Contagem da qtd de regras negativas em que o cliente se enquadrou
tam = 0
for x in listaResultado:
    t = len(x['payloadHomol']['saidas']['regrasNegativas'])
    x['payloadHomol']['saidas']['qtdNegativas'] = t
    if t > tam:
        tam = t

print("tam: "+str(tam))

for y in range (0,tam):
    massa_modelosPriorizacao.atribui_expected(listaResultado,"payloadHomol.saidas.regrasNegativas."+str(y)+".nomeRegra","regrasNegativas#"+str(y+1)+".nomeRegra")
    massa_modelosPriorizacao.atribui_expected(listaResultado,"payloadHomol.saidas.regrasNegativas."+str(y)+".descricao","regrasNegativas#"+str(y+1)+".descRegra")
    
    massa_modelosPriorizacao.atribui_expected(listaResultado,"payloadHomol.saidas.listaLogs."+str(y)+".nome","listaLogs#"+str(y+1)+".nome")
    massa_modelosPriorizacao.atribui_expected(listaResultado,"payloadHomol.saidas.listaLogs."+str(y)+".valor","listaLogs#"+str(y+1)+".valor")




#saidas

massa_modelosPriorizacao.atribui_expected(listaResultado,'payloadHomol.saidas.ghFinal','ghFinal')
massa_modelosPriorizacao.atribui_expected(listaResultado,'payloadHomol.saidas.scoreFinal','scoreFinal')
massa_modelosPriorizacao.atribui_expected(listaResultado,'payloadHomol.saidas.origemModeloApplication','origemModeloApplication')



massa_modelosPriorizacao.get_massa_dm()



tam: 0


,Data-ID,Result-State,Result-Detail,numeroDocumento,dataAdmissaoFopag,dataRegistroPrimeiroContrato,dataRegistro,flagMAT30d,flagMAT35d,flagMAT60d,flagMAT90d,fonteRenda,flagPortabilidade,rendaLiquidaPicpay,consumerID,scoreInternoLongo,scoreInterno,investimentoTotal,flagInvestimentoPrincipalidade,payloadParam.solicitante.flagPrincipalidadeAll,flagPrincipalidade,mesFlagPrincipalidade,flagAllowlist,flagBlocklist,scoreInternoCurto,scoreExterno,flagCartaoCreditoContratado,flagLimiteGarantidoContratado,ghAppcardsBlendInterno,scoreAppcardsBlendInternoOnline,ghAppcardsBlendExterno,scoreAppcardsBlendExternoOnline,ghBlendIntExt,scoreBlendIntExtOnline,ghSerasa,scoreSerasaAlOnline,result:ghFinal,expected:ghFinal,result:scoreFinal,expected:scoreFinal,result:origemModeloApplication,expected:origemModeloApplication
0,1,,,123456789,2002-11-05T17:18:43.888Z,2002-11-05T17:18:43.888Z,2002-11-05T17:18:43.888Z,0,0,0,0,XYZ,0,10000,12345678987654321,999,999,0,0,0,0,0,-99999,-99999,959,959,0,0,B,650,D,500,E,100,F,1000,,B,,650,,Interno + Serasa + SCR
1,2,,,123456789,2002-11-05T17:18:43.888Z,2002-11-05T17:18:43.888Z,2002-11-05T17:18:43.888Z,0,0,0,0,XYZ,0,10000,12345678987654321,999,999,0,0,0,0,0,-99999,-99999,959,959,0,0,B,650,D,500,E,100,F,0,,B,,650,,Interno + Serasa + SCR
2,3,,,123456789,2002-11-05T17:18:43.888Z,2002-11-05T17:18:43.888Z,2002-11-05T17:18:43.888Z,0,0,0,0,XYZ,0,10000,12345678987654321,999,999,0,0,0,0,0,-99999,-99999,959,959,0,0,B,650,D,500,E,100,F,-1,,B,,650,,Interno + Serasa + SCR
3,4,,,123456789,2002-11-05T17:18:43.888Z,2002-11-05T17:18:43.888Z,2002-11-05T17:18:43.888Z,0,0,0,0,XYZ,0,10000,12345678987654321,999,999,0,0,0,0,0,-99999,-99999,959,959,0,0,B,650,D,500,E,100,F,-99999,,B,,650,,Interno + Serasa + SCR
4,5,,,123456789,2002-11-05T17:18:43.888Z,2002-11-05T17:18:43.888Z,2002-11-05T17:18:43.888Z,0,0,0,0,XYZ,0,10000,12345678987654321,999,999,0,0,0,0,0,-99999,-99999,959,959,0,0,B,650,D,500,E,100,missing,1000,,B,,650,,Interno + Serasa + SCR
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
49147,49148,,,123456789,2002-11-05T17:18:43.888Z,2002-11-05T17:18:43.888Z,2002-11-05T17:18:43.888Z,0,0,0,0,XYZ,0,10000,12345678987654321,999,999,0,0,0,0,0,-99999,-99999,959,959,0,0,0,-10,0,-10,0,-1,C,-99999,,C,,-10,,Interno + Serasa + SCR
49148,49149,,,123456789,2002-11-05T17:18:43.888Z,2002-11-05T17:18:43.888Z,2002-11-05T17:18:43.888Z,0,0,0,0,XYZ,0,10000,12345678987654321,999,999,0,0,0,0,0,-99999,-99999,959,959,0,0,0,-10,0,-10,0,-1,0,1000,,missing,,-10,,Interno + Serasa + SCR
49149,49150,,,123456789,2002-11-05T17:18:43.888Z,2002-11-05T17:18:43.888Z,2002-11-05T17:18:43.888Z,0,0,0,0,XYZ,0,10000,12345678987654321,999,999,0,0,0,0,0,-99999,-99999,959,959,0,0,0,-10,0,-10,0,-1,0,0,,missing,,-10,,Interno + Serasa + SCR
49150,49151,,,123456789,2002-11-05T17:18:43.888Z,2002-11-05T17:18:43.888Z,2002-11-05T17:18:43.888Z,0,0,0,0,XYZ,0,10000,12345678987654321,999,999,0,0,0,0,0,-99999,-99999,959,959,0,0,0,-10,0,-10,0,-1,0,-1,,missing,,-10,,Interno + Serasa + SCR


### Conversão da massa para CSV

In [8]:
massa_modelosPriorizacao.get_massa_dm().to_csv('C:/Users/T51657/Desktop/Homologação/Modelos Online/v2/massas/Modulares/massaModelosPriorizacaoV2.csv',index=False)

### Quebra da massa em pedaços

In [9]:

dataframeGrande = massa_modelosPriorizacao.get_massa_dm()
# dataframeGrande

num_dataframes = 4

split_dataframes = np.array_split(dataframeGrande, num_dataframes)

for i, df in enumerate(split_dataframes):
    df.to_csv(f'Massas/Modulares/massaModelosPriorizacaoV2-part{i+1}.csv', index=False)

c:\Users\T51657\AppData\Local\Programs\Python\Python311\Lib\site-packages\numpy\core\fromnumeric.py:59: FutureWarning: 'DataFrame.swapaxes' is deprecated and will be removed in a future version. Please use 'DataFrame.transpose' instead.
  return bound(*args, **kwds)


#### Análises

In [10]:
dataID = 1
print(json.dumps(listaResultado[dataID-1],indent=4))

{
    "marcacaoMesa": "",
    "etapa": "",
    "flagErroBacen": false,
    "flagErroSerasa": false,
    "flagErroAurora": false,
    "flagErroVisaoCliente": false,
    "flagErroReceitaFederal": false,
    "flagErroCredilink": false,
    "limitePoliticaConcessao": -1,
    "mensagemFiltro": "99. Accepted",
    "mensagemFiltroPolitica": "99. Accepted",
    "mensagemFiltroFinal": "99. Accepted",
    "idade": 30,
    "solicitante": {
        "credilink": {
            "situacaoDocumento": "REGULAR",
            "obito": "NAO"
        },
        "limites": {
            "limiteDisponivel": 10000,
            "valorContratadoCartao": 0
        },
        "atrasos": {
            "qtdDiasAtrasoCartao": 0,
            "qtdDiasAtrasoParcelados": 0,
            "qtdDiasAtrasoBNPLMova": 0,
            "qtdDiasAtrasoP2PL": 0
        },
        "restritivosBVS": {
            "vlrMaxRestritivosBVSM1": 0,
            "vlrMaxRestritivosBVSM2": 0,
            "vlrMaxRestritivosBVSM3": 0
        },
    

In [11]:
import pandas as pd
results = []
for caso in listaResultado:
    results.append(caso['payloadHomol']['intermediarias']['regraorigemModeloApplication'])
    
freqzinha = pd.DataFrame(results, columns=['valor'])
freqzinha['Freq'] = freqzinha['valor'].map(freqzinha['valor'].value_counts())
freqzinha = freqzinha.drop_duplicates().reset_index(drop=True)
freqzinha

,valor,Freq
0,1,37120
1,2,8192
2,3,3072
3,4,768


In [12]:
import pandas as pd
results = []
for caso in listaResultado:
    results.append(caso['payloadHomol']['intermediarias']['regraGhFinal'])
    
freqzinha = pd.DataFrame(results, columns=['valor'])
freqzinha['Freq'] = freqzinha['valor'].map(freqzinha['valor'].value_counts())
freqzinha = freqzinha.drop_duplicates().reset_index(drop=True)
freqzinha

,valor,Freq
0,1,24576
1,2,12288
2,3,6144
3,4,3072
4,5,3072


In [13]:
import pandas as pd
results = []
for caso in listaResultado:
    results.append(caso['payloadHomol']['intermediarias']['regraScoreFinal'])
    
freqzinha = pd.DataFrame(results, columns=['valor'])
freqzinha['Freq'] = freqzinha['valor'].map(freqzinha['valor'].value_counts())
freqzinha = freqzinha.drop_duplicates().reset_index(drop=True)
freqzinha

,valor,Freq
0,1,36864
1,2,8192
2,3,3072
3,4,768
4,5,256
